# Internacoes count data
All data was extracted from the SIH database

Import libraries

In [1]:
import pandas as pd
import numpy as np
import glob
root = '../'

Convert raw data to formatted count data

In [2]:
path = root + 'CSV/TabNet/Internacoes/'
all_files = glob.glob(path + "*.csv")
for file in all_files:
    file_name = file.split("/")[-1]
    disease = file_name.split('.csv')[0]
    disease_df = pd.read_csv(path + file_name, sep=';')
    disease_df['MUNCOD'] = disease_df['Município'].str.split(' ', expand=True)[0]
    disease_df = disease_df[disease_df["Município"] != "Total"]
    disease_df.drop(['Município', 'Total'], axis=1, inplace=True)
    disease_df = disease_df.replace("-", 0)
    disease_df = disease_df.astype(int)
    for col in disease_df.columns:
        if(col != "MUNCOD"):
            year = col.split("/")[0]
            if year not in disease_df.columns:
                disease_df[year] = 0
            disease_df[year] = disease_df[year] + disease_df[col]
            disease_df = disease_df.drop(columns=col)
    disease_df.to_csv(root + 'CSV/TabNet/Internacoes_Anual/' + disease +'.csv')

# Internacoes rate data

In [3]:
def drop_if_exists(col_list, df):
    for col in col_list:
        if col in df.columns:
            df = df.drop(columns=col, axis=1)
    return df

In [4]:
path = root + 'CSV/TabNet/Internacoes_Anual/'
all_files = glob.glob(path + "*.csv")
population = pd.read_csv(root + 'CSV/Population/population_08_18.csv', index_col=[0])
for file in all_files:
    file_name = file.split("/")[-1]
    disease = file_name.split(".csv")[0]
    disease_df = pd.read_csv(path + file_name,index_col=0)
    disease_df = drop_if_exists(["2007","2019","2020"],disease_df)
    df = pd.merge(population, disease_df, left_on="MUNCOD", right_on="MUNCOD")
    for column in population.columns:
        if column.split('_')[0] == "POP":
            year = column.split('_')[1]
            if '20' + year in df.columns:
                df['RATE_' + year] = df['20' + year]/df['POP_' + year] * 100000
            df = drop_if_exists(['20' + year, 'POP_' + year], df)
    df.to_csv(root + 'CSV/TabNet/Internacoes_Rate/' + disease + ".csv")
df.head()

,MUNCOD,RATE_18,RATE_17,RATE_16,RATE_15,RATE_14,RATE_13,RATE_12,RATE_11,RATE_10,RATE_09,RATE_08
0,110001,0.000000,0.000000,3.920646,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
1,110009,0.000000,0.000000,0.000000,0.000000,0.0,6.309347,0.000000,0.0,0.0,0.0,0.0
2,110020,0.000000,0.192516,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
3,110120,10.243802,0.000000,0.000000,18.479165,0.0,0.000000,9.801039,0.0,0.0,0.0,0.0
4,110143,0.000000,12.871669,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
